In [ ]:
```
python attack_vector.py \
    --model-path "checkpoint/regressor.pt" \
    --features 5 --maxiter 200 --popsize 800 \
    --val-min -1.0 --val-max 1.0 \
    --samples 100
```

Key command‑line flags
~~~~~~~~~~~~~~~~~~~~~~
--features   : how many components of the 102‑vector the attacker may change
--val-min/max: allowable numeric range for each perturbed component
--error-thr  : relative‑error threshold that defines success (default 0.30)

The script prints a running success rate and a summary at the end.  It produces
no permanent side‑effects; integrate logging or saving results as needed.

Implementation notes
--------------------
* **Genome representation** – For *k* allowed feature edits, an individual has
  length 2k:  (idx₀, value₀, …, idx_{k−1}, value_{k−1}).  Indices are integers
  in [0, input_dim) and are *rounded* inside `perturb_vector`.
* **Bounds** – DE bounds are built as [(0, input_dim‑1), (val_min, val_max)] ⨉ k.
* **Batching** – `perturb_vector` vectorises perturbations so the model sees an
  entire DE population in one forward pass for speed.
* **Data loading** – Provide your own `RegressionDataset` (see stub below) or
  replace it with your loader.  Each `__getitem__` must return (x, y_true).
* **GPU** – Enabled when CUDA is available; otherwise falls back to CPU.

"""
import argparse
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

# Local file copied from the original repo – keep the path if needed.
from differential_evolution import differential_evolution  # noqa: E402


def parse_args():
    p = argparse.ArgumentParser("K‑Feature Differential Evolution Attack (Regression)")
    p.add_argument("--model-path", required=True, help="Path to saved PyTorch model (state_dict or full model).")
    p.add_argument("--features", type=int, default=1, help="How many input indices may be perturbed.")
    p.add_argument("--val-min", type=float, default=-1.0, help="Minimum allowed value for a perturbed component.")
    p.add_argument("--val-max", type=float, default=1.0, help="Maximum allowed value for a perturbed component.")
    p.add_argument("--maxiter", type=int, default=100, help="Maximum DE iterations.")
    p.add_argument("--popsize", type=int, default=400, help="Population‑size multiplier in DE (actual pop = popsize * n_params).")
    p.add_argument("--samples", type=int, default=100, help="How many (x, y) pairs from the dataset to attack.")
    p.add_argument("--error-thr", type=float, default=0.30, help="Relative‑error threshold to declare success.")
    p.add_argument("--batch-size", type=int, default=1, help="Batch size for the data loader (set 1 for simplicity).")
    p.add_argument("--seed", type=int, default=0, help="Random seed for reproducibility.")
    p.add_argument("--device", default="cuda" if torch.cuda.is_available() else "cpu", help="Device to run on.")
    p.add_argument("--verbose", action="store_true", help="Print verbose logs from DE.")
    return p.parse_args()


# ──────────────────────────────────────────────────────────────────────────────
# Utilities
# ──────────────────────────────────────────────────────────────────────────────

def perturb_vector(xs: np.ndarray, base_vec: torch.Tensor) -> torch.Tensor:
    """Apply a batch of perturbations *xs* to *base_vec*.

    Parameters
    ----------
    xs : ndarray, shape (n_pop, 2*k) or (2*k,)
        Genome(s) produced by DE: (idx₀, val₀, …).
    base_vec : Tensor, shape (input_dim,)
        Unperturbed input.

    Returns
    -------
    Tensor, shape (n_pop, input_dim)
        Batch of perturbed inputs ready for the model.
    """
    if xs.ndim == 1:
        xs = np.expand_dims(xs, 0)
    n_pop, genome_len = xs.shape
    k = genome_len // 2

    # Duplicate the base vector n_pop times – stays on CPU for now.
    vecs = base_vec.repeat(n_pop, 1)

    for row, genome in enumerate(xs):
        for j in range(k):
            idx = int(round(genome[2 * j]))  # ensure integer index
            val = genome[2 * j + 1]
            idx_clamped = max(0, min(idx, base_vec.numel() - 1))
            vecs[row, idx_clamped] = val
    return vecs


def relative_l2_error(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """Compute ‖pred ‑ target‖₂ / ‖target‖₂ along dim=1."""
    diff_norm = torch.norm(pred - target, dim=1)
    tgt_norm = torch.norm(target, dim=1).clamp_min(1e-12)
    return diff_norm / tgt_norm


# ──────────────────────────────────────────────────────────────────────────────
# Attack core
# ──────────────────────────────────────────────────────────────────────────────

def de_objective(xs: np.ndarray, base_x: torch.Tensor, y_true: torch.Tensor, model: torch.nn.Module,
                 device: torch.device, maximize: bool = True) -> np.ndarray:
    """Vectorised objective for DE.  Returns *negative* error (to minimise)."""
    perturbed = perturb_vector(xs, base_x).to(device)
    with torch.no_grad():
        y_pred = model(perturbed)
    err = relative_l2_error(y_pred, y_true.to(device)).cpu().numpy()  # shape (n_pop,)
    return -err if maximize else err


def attack_single(base_x: torch.Tensor, y_true: torch.Tensor, model: torch.nn.Module,
                  features: int, error_thr: float, val_min: float, val_max: float,
                  maxiter: int, popsize: int, device: torch.device, verbose: bool = False):
    """Run DE attack on a single (x, y) pair.  Returns (success, best_genome)."""
    input_dim = base_x.numel()
    bounds = []
    for _ in range(features):
        bounds.append((0, input_dim - 1))      # index
        bounds.append((val_min, val_max))      # new value

    # Pop‑multiplier like original code (total pop = popsize * n_params)
    popmul = max(1, popsize)

    predict_fn = lambda xs: de_objective(xs, base_x, y_true, model, device, True)

    def callback_fn(genome, convergence):
        # Early stop if success achieved
        perturbed = perturb_vector(genome, base_x).to(device)
        with torch.no_grad():
            err = relative_l2_error(model(perturbed), y_true.to(device))[0].item()
        if verbose:
            print(f"  Current best error = {err:.3f}")
        return err > error_thr

    # Optional initial population: random indices + gaussian values
    n_params = 2 * features
    inits = np.zeros((popmul * n_params, n_params))
    for row in inits:
        for f in range(features):
            row[2 * f] = np.random.randint(0, input_dim)
            row[2 * f + 1] = np.random.uniform(val_min, val_max)

    result = differential_evolution(
        predict_fn,
        bounds,
        maxiter=maxiter,
        popsize=popmul,
        recombination=1.0,
        atol=-1,
        init=inits,
        polish=False,
        callback=callback_fn,
        disp=False,
    )

    # Evaluate final error
    final_vec = perturb_vector(result.x, base_x).to(device)
    with torch.no_grad():
        final_err = relative_l2_error(model(final_vec), y_true.to(device))[0].item()

    success = final_err > error_thr
    return success, result.x, final_err


# ──────────────────────────────────────────────────────────────────────────────
# Example dataset stub – replace with your data loader
# ──────────────────────────────────────────────────────────────────────────────
class RegressionDataset(Dataset):
    """Dummy dataset generating random vectors – replace with real data."""
    def __init__(self, n_samples: int = 1000, seed: int = 0):
        rng = np.random.RandomState(seed)
        self.x = torch.from_numpy(rng.uniform(-1, 1, (n_samples, 102))).float()
        self.y = torch.from_numpy(rng.uniform(-1, 1, (n_samples, 1000))).float()

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]


# ──────────────────────────────────────────────────────────────────────────────
# Main driver
# ──────────────────────────────────────────────────────────────────────────────

def main():
    args = parse_args()
    torch.manual_seed(args.seed)
    np.random.seed(args.seed)

    device = torch.device(args.device)

    # Load model – works for either full pickled model or state_dict
    model_obj = torch.load(args.model_path, map_location=device)
    model = model_obj if isinstance(model_obj, torch.nn.Module) else None
    if model is None:
        raise ValueError("Provide a saved *model object*, not just a state_dict – ease of use.")
    model.eval()

    # TODO: replace RegressionDataset with your own data loader
    dataset = RegressionDataset(n_samples=args.samples, seed=args.seed)
    loader = DataLoader(dataset, batch_size=args.batch_size, shuffle=True)

    successes = 0
    total = 0

    for i, (x, y) in enumerate(loader):
        if total >= args.samples:
            break
        x = x.squeeze(0).to(device)  # ensure shape (input_dim,)
        y = y.squeeze(0).to(device)

        success, genome, err = attack_single(
            x.cpu(), y.cpu(), model,
            features=args.features,
            error_thr=args.error_thr,
            val_min=args.val_min,
            val_max=args.val_max,
            maxiter=args.maxiter,
            popsize=args.popsize,
            device=device,
            verbose=args.verbose,
        )

        total += 1
        successes += int(success)

        status = "✔" if success else "✘"
        print(f"[{i+1:03d}] {status}  final error = {err:.3f}  success‑rate = {successes / total:.3%}")

    print(f"\nFinal success rate over {total} samples: {successes / total:.2%}")


if __name__ == "__main__":
    main()
